In [0]:
import csv
import pandas as pd
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Bidirectional, LSTM, Dense, Dropout, ThresholdedReLU, Flatten
from keras.models import Sequential
from tensorflow.keras import Model
tf.test.gpu_device_name() 

'/device:GPU:0'

In [0]:
# Code to read csv file into Colaboratory:
from google.colab import drive
drive.mount('/content/drive')
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# downloaded = drive.CreateFile({'id':'1LAXYuJapp6u6shheA3ciIy67GrOQdlKs'}) 
# downloaded.GetContentFile('q2_revised.csv')  
df2 = pd.read_csv('/content/drive/Shared drives/Hackathon_new/q2_revised.csv', encoding='latin1')
# Dataset is now stored in a Pandas Dataframe
gt2 = pd.read_excel('/content/drive/Shared drives/Hackathon_new/q2_answer.xlsx')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
df2.head()

,No.,Time,Source,Destination,Protocol,Length,Text item,Transaction ID,Name,Time.1,Reply code,Response,Primary name server,Info
0,1,0.000000,183.213.21.48,166.111.8.29,DNS,73,â\234\223,0x2719,weilaigou.com,NaN,NaN,Message is a query,NaN,Standard query 0x2719 A weilaigou.com
1,2,0.044210,8.8.8.8,101.6.69.132,DNS,102,â\234\223,0x51ae,taobao.com,NaN,No error,Message is a response,NaN,Standard query response 0x51ae A taobao.com A ...
2,3,0.069832,8.8.8.8,101.6.128.60,DNS,68,â\234\223,0xf102,sohu.com,NaN,Server failure,Message is a response,NaN,Standard query response 0xf102 Server failure ...
3,4,0.090890,144.217.10.132,166.111.124.17,DNS,79,â\234\223,0xbde3,hrsa.gov,NaN,NaN,Message is a query,NaN,Standard query 0xbde3 ANY hrsa.gov OPT
4,5,0.147890,183.173.94.53,8.8.8.8,DNS,70,â\234\223,0xb6ce,wahom.site,NaN,NaN,Message is a query,NaN,Standard query 0xb6ce A wahom.site


In [0]:
gt2

,domain,family
0,z54f71fcb61e49158f3648236556b04eb1.so,1
1,l31f5dcf7f398d2b94cd9155ee392c701b.so,1
2,d442a48406424819e686a5d1fde7c602ea.so,1
3,zba909e91846ce27aff7902f9228ee40ea.so,1
4,p8f5889e854374cf8e70e91f456807f83f.so,1
...,...,...
11090,gdwrhiivyt.cc,10
11091,hyirnrnetku.cc,10
11092,isauvvu.cc,10
11093,vddiuiygf.cc,10


In [0]:
def extractDNS(info):
    info = info.split(' ')
    for s in info:
        if '.' in s:
            return s
    
    return ''

In [0]:
df2['dns'] = df2.Info.apply(extractDNS)

# Part 1: classify the malicious DNS and benign DNS

### prepare the data
input is the DNS string
output is 1 or 0. 1 means malicious and 0 means benign

In [0]:
data_x = df2.dns.unique()

In [0]:
# check all the ground truth dns are in the train_x
x_set = set(data_x)
for dns in gt2.domain:
    if dns not in x_set:
        print(dns)

In [0]:
# gt_set = set(gt2.domain)
# data_y = []
# for dns in data_x:
#     if dns in gt_set:
#         data_y.append(1)
#     else:
#         data_y.append(0)

In [0]:
# sum(data_y)

In [0]:
# len(gt_set)

In [0]:
# there are some 42 dns are both in family 1 and 2. I guess
# they are in 2 since they end with '.ws'. 
gt_set = set()
repeat_dns = []
for dns in gt2.domain:
    if dns not in gt_set:
        gt_set.add(dns)
    else:
        repeat_dns.append(dns)

In [0]:
drop_idx = [] #drop index
for dns in repeat_dns:
    drop_idx.append(gt2[(gt2.domain==dns) & (gt2.family == 1)].index[0])

ngt2 = gt2.drop(drop_idx) #new answers
ngt2

,domain,family
0,z54f71fcb61e49158f3648236556b04eb1.so,1
1,l31f5dcf7f398d2b94cd9155ee392c701b.so,1
2,d442a48406424819e686a5d1fde7c602ea.so,1
3,zba909e91846ce27aff7902f9228ee40ea.so,1
4,p8f5889e854374cf8e70e91f456807f83f.so,1
...,...,...
11090,gdwrhiivyt.cc,10
11091,hyirnrnetku.cc,10
11092,isauvvu.cc,10
11093,vddiuiygf.cc,10


In [0]:
gt_set = set(ngt2.domain)
data_y = []
for dns in data_x:
    if dns in gt_set:
        idx = ngt2.index[ngt2['domain'] == dns].tolist()[0] #get index where domain = dns
        family = ngt2.at[int(idx), 'family'] #get family value from index
        data_y.append(family)
    else:
        data_y.append(0) #0 if not DGA
#data_y

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 3,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 8,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 10,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,

In [0]:
p1_data = pd.DataFrame(list(zip(data_x, data_y)), columns =['dns', 'family'])
p1_data = p1_data.sample(frac=1, random_state=2020)
p1_data

,dns,family
47677,scylgroup.com,0
48162,effilnuds.com,0
28840,lovebookonline.com,0
4204,wwww91k.com,0
30276,93c53.com,0
...,...,...
18523,sousoucili.com,0
44739,ngmc.com,0
47734,weborama.fr,0
40259,mg4150.com,0


In [0]:
p1_data['dnsLength'] = p1_data.dns.apply(lambda x: len(x))

In [0]:
def removeEnding(domain):
  domain = domain.split(".", 1)[0]
  return domain

In [0]:
p1_data['dns'] = p1_data.dns.apply(lambda x: removeEnding(x))


NameError: ignored

In [0]:
p1_data

,dns,family,dnsLength
47677,scylgroup,0,13
48162,effilnuds,0,13
28840,lovebookonline,0,18
4204,wwww91k,0,11
30276,93c53,0,9
...,...,...,...
18523,sousoucili,0,14
44739,ngmc,0,8
47734,weborama,0,11
40259,mg4150,0,10


In [0]:
def dnsToMatrix(dns):
    maxl = 63
#     demision = 128
    asc = [ord(i.lower()) for i in dns]
    asc = [0] * (maxl-len(asc)) + asc
    asc = np.array(asc)
#     res = np.zeros((demision, maxl))
#     res[asc, np.arange(asc.size),] = 1
    
    return asc

In [0]:
np_x = []
for dns in p1_data.dns:
    np_x.append(dnsToMatrix(dns))
np_x = np.stack(np_x, axis=0)

In [0]:
np_x.shape
np_x

array([[  0,   0,   0, ..., 111, 117, 112],
       [  0,   0,   0, ..., 117, 100, 115],
       [  0,   0,   0, ..., 105, 110, 101],
       ...,
       [  0,   0,   0, ...,  97, 109,  97],
       [  0,   0,   0, ...,  49,  53,  48],
       [  0,   0,   0, ..., 118, 115, 113]])

In [0]:
def familyToMatrix(family): #turns 8 into [0,0,0,0,0,0,0,0,1,0,0] index 8 becomes 1
  out = [0]*11
  out[family] = 1
  return np.array(out)

In [0]:
np_y = []
for family in p1_data.family:
  np_y.append(familyToMatrix(family)) #1 hot encoding
np_y = np.stack(np_y, axis=0)
print(np_y.shape)
np_y

(56625, 11)


array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [0]:
n = len(p1_data)
train_ratio = 0.8 
val_ratio = 0.1
test_ratio = 0.1

train_x = np_x[:int(n*train_ratio),:]
train_y = np_y[:int(n*train_ratio),:]

val_x = np_x[int(n*train_ratio):int(n*(train_ratio+val_ratio)),:]
val_y = np_y[int(n*train_ratio):int(n*(train_ratio+val_ratio)),:]

test_x = np_x[int(n*(train_ratio+val_ratio)):,:]
test_y = np_y[int(n*(train_ratio+val_ratio)):,:]


In [0]:
test_x.shape
#test_y.shape

(5663, 63)

###MIT MODEL

In [0]:
main_input = Input(shape=(63,), dtype='int32', name='input')
embedding = Embedding(input_dim=128, output_dim=128, input_length=63)(main_input)
conv = Conv1D(filters=128, kernel_size=4, padding='same', activation='relu', strides=1)(embedding)
max_pool = MaxPooling1D(pool_size=2, padding='same')(conv) 
encode = LSTM(64, return_sequences=False)(max_pool)
output = Dense(11, activation='softmax')(encode) #11 outputs

model = Model(inputs=main_input, outputs=output)
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 63)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 63, 128)           16384     
_________________________________________________________________
conv1d (Conv1D)              (None, 63, 128)           65664     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 32, 128)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 11)                715       
Total params: 132,171
Trainable params: 132,171
Non-trainable params: 0
_______________________________________________________

In [0]:
history = model.fit(train_x, train_y,
                    batch_size=64,
                    epochs=10,
                    validation_data=(val_x, val_y))

Epoch 1/10
708/708 [==============================] - 6s 8ms/step - loss: 0.7476 - accuracy: 0.8036 - val_loss: 0.7006 - val_accuracy: 0.8004
Epoch 2/10
708/708 [==============================] - 5s 7ms/step - loss: 0.6716 - accuracy: 0.8055 - val_loss: 0.6821 - val_accuracy: 0.8011
Epoch 3/10
708/708 [==============================] - 5s 7ms/step - loss: 0.6553 - accuracy: 0.8053 - val_loss: 0.6727 - val_accuracy: 0.7992
Epoch 4/10
708/708 [==============================] - 5s 7ms/step - loss: 0.6447 - accuracy: 0.8048 - val_loss: 0.6644 - val_accuracy: 0.7971
Epoch 5/10
708/708 [==============================] - 5s 7ms/step - loss: 0.6366 - accuracy: 0.8049 - val_loss: 0.6608 - val_accuracy: 0.8018
Epoch 6/10
708/708 [==============================] - 5s 7ms/step - loss: 0.6272 - accuracy: 0.8047 - val_loss: 0.6614 - val_accuracy: 0.8010
Epoch 7/10
708/708 [==============================] - 5s 7ms/step - loss: 0.6192 - accuracy: 0.8063 - val_loss: 0.6546 - val_accuracy: 0.7994
Epoch 

In [0]:
test_loss, test_accuracy = model.evaluate(test_x, test_y)
print(test_accuracy)

177/177 [==============================] - 1s 3ms/step - loss: 0.6521 - accuracy: 0.7998
0.7997527718544006


In [0]:
y = model.predict(test_x)

In [0]:
threshold = 0.9

# End Game Model

In [0]:
main_input = Input(shape=(63, ), dtype='int32', name='main_input')
embedding = Embedding(input_dim=128, output_dim=128, input_length=63)(main_input)
lstm = LSTM(128, return_sequences=False)(embedding)
drop = Dropout(0.5)(lstm)
output = Dense(11, activation='softmax')(drop)

EndgameModel = Model(inputs=main_input, outputs=output)
EndgameModel.summary()
EndgameModel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      [(None, 63)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 63, 128)           16384     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 11)                1419      
Total params: 149,387
Trainable params: 149,387
Non-trainable params: 0
_________________________________________________________________


In [0]:
historyEndgame = EndgameModel.fit(train_x, train_y,
                    batch_size=64,
                    epochs=10,
                    validation_data=(val_x, val_y))

Epoch 1/10
708/708 [==============================] - 6s 8ms/step - loss: 0.7949 - accuracy: 0.8016 - val_loss: 0.7406 - val_accuracy: 0.8020
Epoch 2/10
708/708 [==============================] - 6s 8ms/step - loss: 0.7134 - accuracy: 0.8026 - val_loss: 0.6970 - val_accuracy: 0.8025
Epoch 3/10
708/708 [==============================] - 6s 8ms/step - loss: 0.6845 - accuracy: 0.8026 - val_loss: 0.6821 - val_accuracy: 0.7964
Epoch 4/10
708/708 [==============================] - 6s 8ms/step - loss: 0.6735 - accuracy: 0.8023 - val_loss: 0.6766 - val_accuracy: 0.7969
Epoch 5/10
708/708 [==============================] - 6s 8ms/step - loss: 0.6649 - accuracy: 0.8031 - val_loss: 0.6748 - val_accuracy: 0.7992
Epoch 6/10
708/708 [==============================] - 6s 8ms/step - loss: 0.6602 - accuracy: 0.8036 - val_loss: 0.6692 - val_accuracy: 0.7967
Epoch 7/10
708/708 [==============================] - 6s 8ms/step - loss: 0.6541 - accuracy: 0.8039 - val_loss: 0.6658 - val_accuracy: 0.8018
Epoch 

In [0]:
test_loss, test_accuracy = model.evaluate(test_x, test_y)
print(test_accuracy)

177/177 [==============================] - 1s 3ms/step - loss: 0.6521 - accuracy: 0.7998
0.7997527718544006


#NYU Model

In [0]:

##################################### didn't run these 2 models
main_input = Input(shape=(63, ), dtype='int32', name='main_input') 
embedding = Embedding(input_dim=128, output_dim=128, input_length=63)(main_input)
conv1 = Conv1D(filters=128, kernel_size=3, padding='same', strides=1)(embedding)
thresh1 = ThresholdedReLU(1e-6)(conv1)
max_pool1 = MaxPooling1D(pool_size=2, padding='same')(thresh1)
conv2 = Conv1D(filters=128, kernel_size=2, padding='same', strides=1)(max_pool1)
thresh2 = ThresholdedReLU(1e-6)(conv2)
max_pool2 = MaxPooling1D(pool_size=2, padding='same')(thresh2)
flatten = Flatten()(max_pool2)
fc = Dense(64)(flatten)
thresh_fc = ThresholdedReLU(1e-6)(fc)
drop = Dropout(0.5)(thresh_fc)
output = Dense(11, activation='softmax')(drop)

NyuModel = Model(inputs=main_input, outputs=output)
NyuModel.summary()
NyuModel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      [(None, 63)]              0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 63, 128)           16384     
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 63, 128)           49280     
_________________________________________________________________
thresholded_re_lu_6 (Thresho (None, 63, 128)           0         
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 32, 128)           0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 32, 128)           32896     
_________________________________________________________________
thresholded_re_lu_7 (Thresho (None, 32, 128)           0   

In [0]:
historyNyuModel = NyuModel.fit(train_x, train_y,
                    batch_size=64,
                    epochs=10,
                    validation_data=(val_x, val_y))

Epoch 1/10
708/708 [==============================] - 4s 6ms/step - loss: 0.7882 - accuracy: 0.8034 - val_loss: 0.7172 - val_accuracy: 0.8018
Epoch 2/10
708/708 [==============================] - 4s 6ms/step - loss: 0.7030 - accuracy: 0.8038 - val_loss: 0.6823 - val_accuracy: 0.8018
Epoch 3/10
708/708 [==============================] - 4s 6ms/step - loss: 0.6828 - accuracy: 0.8059 - val_loss: 0.6724 - val_accuracy: 0.8017
Epoch 4/10
708/708 [==============================] - 4s 6ms/step - loss: 0.6685 - accuracy: 0.8048 - val_loss: 0.6717 - val_accuracy: 0.8020
Epoch 5/10
708/708 [==============================] - 4s 6ms/step - loss: 0.6571 - accuracy: 0.8045 - val_loss: 0.6732 - val_accuracy: 0.8015
Epoch 6/10
708/708 [==============================] - 4s 6ms/step - loss: 0.6459 - accuracy: 0.8050 - val_loss: 0.6693 - val_accuracy: 0.8020
Epoch 7/10
708/708 [==============================] - 4s 6ms/step - loss: 0.6365 - accuracy: 0.8055 - val_loss: 0.6690 - val_accuracy: 0.7995
Epoch 

#MLP MODEL

In [0]:
main_input = Input(shape=(63, ), name='main_input')
dense = Dense(128, activation='relu')(main_input)
output = Dense(11, activation='sigmoid')(dense)

model = Model(inputs=main_input, outputs=output)

mlp = Model(inputs=main_input, outputs=output)
mlp.summary()
mlp.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      [(None, 63)]              0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               8192      
_________________________________________________________________
dense_9 (Dense)              (None, 11)                1419      
Total params: 9,611
Trainable params: 9,611
Non-trainable params: 0
_________________________________________________________________


In [0]:
historyMlp = mlp.fit(train_x, train_y,
                    batch_size=64,
                    epochs=50,
                    validation_data=(val_x, val_y))

Epoch 1/50
708/708 [==============================] - 2s 3ms/step - loss: 0.3694 - accuracy: 0.7570 - val_loss: 0.1516 - val_accuracy: 0.7919
Epoch 2/50
708/708 [==============================] - 2s 3ms/step - loss: 0.1407 - accuracy: 0.7883 - val_loss: 0.1509 - val_accuracy: 0.8031
Epoch 3/50
708/708 [==============================] - 2s 3ms/step - loss: 0.1298 - accuracy: 0.7917 - val_loss: 0.1318 - val_accuracy: 0.7849
Epoch 4/50
708/708 [==============================] - 2s 3ms/step - loss: 0.1243 - accuracy: 0.7948 - val_loss: 0.1269 - val_accuracy: 0.8001
Epoch 5/50
708/708 [==============================] - 2s 3ms/step - loss: 0.1213 - accuracy: 0.7951 - val_loss: 0.1162 - val_accuracy: 0.7958
Epoch 6/50
708/708 [==============================] - 2s 2ms/step - loss: 0.1179 - accuracy: 0.7964 - val_loss: 0.1218 - val_accuracy: 0.7775
Epoch 7/50
708/708 [==============================] - 2s 2ms/step - loss: 0.1164 - accuracy: 0.7986 - val_loss: 0.1245 - val_accuracy: 0.8011
Epoch 